# Visualising results of GAIL hyperparmeter sweep

In [ ]:
%matplotlib inline

In [ ]:
import cloudpickle
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from skopt.plots import plot_evaluations, plot_objective

In [ ]:
sns.set(context='notebook', style='darkgrid')

In [ ]:
pickle_path = glob.glob('../ray-tune-results/ray_tune_trial/search-alg-*.pkl')[-1]
with open(pickle_path, 'rb') as fp:
    _, skopt_alg = cloudpickle.load(fp)
skopt_res = skopt_alg.get_result()

In [ ]:
_ = plot_evaluations(skopt_res)

In [ ]:
_ = plot_objective(skopt_res, n_samples=40) # , minimum='expected_minimum_random', n_minimum_search=1000)

In [ ]:
n_results = len(skopt_res.func_vals)
sns.distplot(skopt_res.func_vals, rug=True, norm_hist=False, kde=False, bins=10 if n_results >= 20 else None)
plt.title(f"Negative final score distribution from {n_results} runs (lower = better)")
plt.xlabel("Negative final score")
plt.ylabel("Frequency")
plt.show()

In [ ]:
import skopt
import matplotlib
import jupyterlab
print('skopt:', skopt.__version__)
print('seaborn:', sns.__version__)
print('matplotlib:', matplotlib.__version__)
print('jupyterlab:', jupyterlab.__version__)

In [ ]:
from skopt.utils import expected_minimum_random_sampling
skopt_min = expected_minimum_random_sampling(skopt_res, n_random_starts=100000)
print("minimum from random sampling:", skopt_min)

In [ ]:
skopt_minima = []
for i in range(10):
    skopt_min = expected_minimum_random_sampling(skopt_res, n_random_starts=1000000)
    skopt_minima.append(skopt_min[0])
print('Minimum from random sampling:')
for idx, dim in enumerate(skopt_res.space.dimensions):
    name = dim.name
    values = [m[idx] for m in skopt_minima]
    if isinstance(values[0], float):
        stringified = [f'{v:.3g}' for v in values]
    else:
        stringified = map(str, values)
    min_str = f'  {name} = [{", ".join(stringified)}]'
    print(min_str)

In [ ]:
thresh = 0.7
good_inds, = np.nonzero(skopt_res.func_vals <= -thresh)
for conf_num, good_ind in enumerate(good_inds, start=1):
    print(f"Good config at index {good_ind} ({conf_num}/{len(good_inds)}), thresh {thresh}:")
    # TODO: print function value here too
    for dimension, value in zip(skopt_res.space.dimensions, skopt_res.x_iters[good_ind]):
        print(f'    {dimension.name} = {value}')
        
print(f'Amalgamated "good" configs at thresh {thresh}:')
for dim_idx, dimension in enumerate(skopt_res.space.dimensions):
    values = [skopt_res.x_iters[i][dim_idx] for i in good_inds]
    if isinstance(values[0], float):
        values_str = f"[{', '.join('%.3g' % v for v in values)}]"
    else:
        values_str = str(values)
    print(f'    {dimension.name} = {values_str}')